<a href="https://colab.research.google.com/github/ednavivianasegura/ERAP_Curso_R/blob/main/ERAP_R_Course_Modulo_IV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Título: Manipulación de datos con R - Módulo IV (Exploración de datos y Estadística descriptiva)

Autor(es): Edna Viviana Segura Alvarado - Hans Mauricio Carrillo Hernández

Fecha: 2025-06

Institución: Universidad de La Rioja

# Estadística Descriptiva

La estadística descriptiva se encarga de recolectar, organizar, representar y resumir la información contenida en un conjunto de datos. Su objetivo es **describir las características fundamentales** de una o más variables, sin llegar a realizar inferencias o generalizaciones.

---

## Clasificación de las variables

| Tipo de variable     | Subtipo              | Ejemplo                            |
|----------------------|----------------------|------------------------------------|
| **Cualitativa**      | Nominal              | Sexo (masculino/femenino)         |
|                      | Ordinal              | Nivel educativo (bajo/medio/alto) |
| **Cuantitativa**     | Discreta             | Número de hijos (0, 1, 2…)         |
|                      | Continua             | Edad, ingreso mensual, peso       |

---

### Enfoque de análisis:

| Enfoque           | Descripción |
|-------------------|-------------|
| **Univariado**    | Análisis de una sola variable. Se explora su distribución, tendencia central, dispersión y forma. |
| **Bivariado**     | Análisis conjunto de dos variables para identificar relaciones o asociaciones. |
| *(Multivariado)*  | Análisis simultáneo de tres o más variables (fuera del alcance de esta sección). |

---

## Estadística Descriptiva Univariada

1. Para Variables **Cualitativas**:

Se resumen usando tablas de frecuencia:

- **Frecuencia absoluta**: número de observaciones por categoría.
- **Frecuencia relativa**: proporción (o porcentaje) respecto al total.
- **Representaciones gráficas**: gráfico de barras, gráfico de torta.

2. Para Variables **Cuantitativas**:

Se utilizan:

- **Medidas de tendencia central**: media, mediana, moda.
- **Medidas de dispersión**: rango, varianza, desviación estándar, coeficiente de variación.
- **Medidas de forma**: asimetría (skewness), curtosis (kurtosis).
- **Visualizaciones**: histogramas, diagramas de caja (boxplots), Q-Q plots.

---

## Estadística Descriptiva Bivariada

1. Dos Variables cualitativas:
- Tablas de contingencia
- Gráficos de mosaico o de barras agrupadas
- Medidas de asociación (por ejemplo, prueba Chi²)

2. Una Variable Cualitativa y una Cuantitativa:
- Promedios o medianas por grupo
- Boxplots por categoría
- ANOVA exploratoria

3. Dos Variables Cuantitativas:
- Nube de puntos (scatterplot)
- Coeficiente de correlación (Pearson, Spearman)
- Covarianza
- Recta de regresión simple

---



In [1]:
# @title Librerías
# Paquetes necesarios

# Función para instalar si no está instalado
instalar_si_no <- function(paquete) {
    if (!requireNamespace(paquete, quietly = TRUE)) {
    install.packages(paquete, repos = "https://cloud.r-project.org")
  }
  library(paquete, character.only = TRUE)
}

# Lista de paquetes a verificar
paquetes <- c("mlbench","forcats","VIM","knitr","e1071","tidyr","dplyr","ggplot2","e1071", "visdat","reshape2", "janitor")


# Instalar y cargar todos
invisible(lapply(paquetes, instalar_si_no))

# Instalar librería empire
remotes::install_github("davidbiol/empire")



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘cowplot’, ‘Deriv’, ‘microbenchmark’, ‘Rdpack’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘zoo’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘DEoptimR’, ‘lmtest’, ‘proxy’, ‘RcppEigen’, ‘colorspace’, ‘car’, ‘robustbase’, ‘sp’, ‘vcd’, ‘e1071’, ‘laeken’, ‘ranger’


Loading required package: colorspace

Loading required package: grid

VIM is ready to use.


Suggestions and bug-reports can be submitted at: https://github.com/statistikat/VIM/issues


Attaching package: ‘VIM’


The following object is masked from ‘package:datasets’:

    sleep



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is

iterators (NA -> 1.0.14 ) [CRAN]
shape     (NA -> 1.4.6.1) [CRAN]
foreach   (NA -> 1.5.2  ) [CRAN]
glmnet    (NA -> 4.1-9  ) [CRAN]


Installing 4 packages: iterators, shape, foreach, glmnet

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/RtmpYRULK8/remotes2ab35c20d27/davidbiol-empire-af6dc96/DESCRIPTION’ ... OK
* preparing ‘empire’:
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
Omitted ‘LazyData’ from DESCRIPTION
* building ‘empire_0.1.0.tar.gz’



Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# Datos

Utilizaremos nuevamente el conjunto Sleep de la librería VIM trabajado durante el módulo II.



In [2]:
# Cargar dataset 'sleep'
data("sleep")
sleep

BodyWgt,BrainWgt,NonD,Dream,Sleep,Span,Gest,Pred,Exp,Danger
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
6654.000,5712.0,NA,NA,3.3,38.6,645,3,5,3
1.000,6.6,6.3,2.0,8.3,4.5,42,3,1,3
3.385,44.5,NA,NA,12.5,14.0,60,1,1,1
0.920,5.7,NA,NA,16.5,NA,25,5,2,3
2547.000,4603.0,2.1,1.8,3.9,69.0,624,3,5,4
10.550,179.5,9.1,0.7,9.8,27.0,180,4,4,4
0.023,0.3,15.8,3.9,19.7,19.0,35,1,1,1
160.000,169.0,5.2,1.0,6.2,30.4,392,4,5,4
3.300,25.6,10.9,3.6,14.5,28.0,63,1,2,1


## Análisis univariado:


### Medidas de Tendencia Central, posición y coeficiente de variación:

Utilizaremos la función summarise de dplyr para encontrar las medidas de tendencia central y de posición de la variable **BodyWgt**, que no tiene datos faltantes:



In [3]:
# Solo seleccionamos variables numéricas y eliminamos NAs (aunque de antemanos sabemos que no tiene NAs)



In [4]:
# Calcular resumen estadístico de la variable BodyWgt



In [5]:
# Calcular moda con count y slice_max



¿Qué nos dicen estos valores?:

| Estadístico                     | Valor      | Interpretación                                                                                                                          |
| ------------------------------- | ---------- | --------------------------------------------------------------------------------------------------------------------------------------- |
| **media**                       | 198.79     | El peso promedio es muy alto, pero está distorsionado por valores extremos (como elefantes o ballenas).                                 |
| **mediana**                     | 3.34       | El valor central realista: la mitad de los animales pesa menos de 3.34 kg. Mucho menor que la media → **asimetría positiva**.           |
| **varianza**                    | 808485.13  | Muestra una altísima dispersión de los pesos. Valores grandes como 6654 lo elevan fuertemente.                                          |
| **desviación típica**           | 899.16     | Indica gran dispersión respecto a la media. Al igual que la varianza, está **muy afectada por los valores extremos**.                   |
| **mínimo**                      | 0.00       | Existen animales extremadamente ligeros (posiblemente error de redondeo o especie muy pequeña).                                         |
| **máximo**                      | 6654.00    | Hay especies con más de 6 toneladas de peso. Confirma la existencia de outliers muy altos.                                              |
| **rango**                       | 6654.00    | Es la diferencia entre el máximo y el mínimo. Muy alto → **alta variabilidad global**.                                                  |
| **IQR (rango intercuartílico)** | 47.60      | El 50% central de los datos está entre 0.6 y 48.2 kg. Mucho más representativo que el rango total.                                      |
| **Q1**                          | 0.60       | El 25% de los animales pesa menos de 0.6 kg.                                                                                            |
| **Q3**                          | 48.20      | El 75% pesa menos de 48.2 kg.                                                                                                           |
| **P10**                         | 0.08       | El 10% de los animales pesa menos de 80 gramos.                                                                                         |
| **P90**                         | 205.50     | El 90% pesa menos de 205.5 kg.                                                                                                          |
| **P95**                         | 518.20     | El 95% pesa menos de 518.2 kg.                                                                                                          |
| **LI (límite inferior)**        | -70.80     | No se usa aquí porque ningún peso puede ser negativo.                                                                                   |
| **LS (límite superior)**        | 119.61     | Todo valor mayor que este es considerado outlier según el criterio de 1.5 \* IQR.                                                       |
| **total**                       | 62.00      | Se analizaron 62 observaciones con datos completos para `BodyWgt`.                                                                      |
| **outliers**                    | 10.00      | Hay 10 valores considerados **atípicos**.                                                                                               |
| **proporción de outliers**      | 0.16 (16%) | Es **una proporción elevada**: casi 1 de cada 6 observaciones es un outlier.                                                            |
| **asimetría**                   | 6.25       | Muy por encima de 0 → **fuerte asimetría positiva**. Hay una larga cola hacia la derecha.                                               |
| **curtosis**                    | 40.60      | Muy superior a 3 → distribución **leptocúrtica**: concentrada alrededor de la mediana, con colas muy pesadas (muchos valores extremos). |
| **Coeficiente de variación**| 452.32%|La desviación típica es más de 4 veces la media.|


Validación gráfica:

In [6]:
#Diagrama de barras de la variable BodyWht



In [7]:
#Boxplot de la bariable BodyWgt


In [8]:
#Boxplot sin valores extremos y modificando los límites del eje


La variable **BodyWgt** tiene una distribución altamente asimétrica y con colas pesadas.

Las medidas robustas (mediana, IQR) son mucho más representativas que la media o la desviación típica.

Un 16% de los datos son outliers, todos por encima del límite superior.

Se recomienda para el análisis:
- Trabajar con logaritmos o escalas logarítmicas.
- Reportar mediana en lugar de media.

In [9]:
# Filtrar valores mayores que 0 antes de aplicar el log
# Crear una nueva columna con logaritmo base 10



In [10]:
# Calcular resumen estadístico de la variable BodyWgt



### Tablas para informes

Usamos de la librería knitr de R, la función kable que permite crear tablas en documentos R Markdown.

In [11]:
# Mostrar tabla


In [12]:
#histograma



#boxplot





¿La variable **BodyWgt** sigue una distribución normal?

In [13]:
# Crear gráfico: Histograma de la variable BodyWgt con curva normal superpuesta







In [14]:
#Diagramas q-q plot:





In [15]:
#Test de normalidad




¿y la variable **log_BodyWgt**?

In [16]:
# Crear gráfico: Histograma del logaritmo de BodyWgt con una curva normal superpuesta





In [17]:
#qqplot varaible log



In [18]:
#test de normalidad






Podemos así ver la importancia del análisis inicial y de la conveniencia de usar transformaciones de las variables.

## Análisis de Correlación y Covarianza

Explorar relaciones entre variables cuantitativas, identificar patrones lineales y cuantificar la fuerza y dirección de esas relaciones mediante medidas como correlación y covarianza.

- **Correlación de Pearson (cor())**

Variante normalizada de la covarianza.

Valores que oscilan entre -1 y 1:

* +1: correlación perfecta positiva
* 0: sin relación lineal
* -1: correlación perfecta negativa

Supone linealidad y normalidad de las variables.

- **Correlación de Spearman**

Se basa en rangos → no asume normalidad ni linealidad.
Útil para datos ordinales o relaciones no lineales monótonas.

- **Covarianza (cov())**

Mide la tendencia conjunta de dos variables a aumentar o disminuir.

* Valor positivo: las variables tienden a crecer juntas.
* Valor negativo: una aumenta y la otra disminuye.

Muy sensible a la escala de las variables




### Correlación:

Comenzamos con el análisis de correlación

In [19]:
# Comenzamos seleccionando solo variables numéricas completas
# recordemos que tenemos que convertir las trews variables pred, exp y danger en factores:

# Conversión de variables numéricas a factores (categorías)




#Para no perder los datos, vamos a imputar con ridge, como hicimos en el módulo II:



# Aplicar imputación por regresión lineal penalizada (ridge)




In [20]:
# Correlación de Pearson




In [21]:
# Correlación de Spearman




| Característica      | **Correlación de Pearson**            | **Correlación de Spearman**                                  |
| ------------------- | ------------------------------------- | ------------------------------------------------------------ |
| Tipo de relación    | Lineal                                | Monótona (creciente o decreciente, no necesariamente lineal) |
| Datos esperados     | Cuantitativos, normales, sin outliers | Cuantitativos u ordinales, **no requiere normalidad**        |
| Sensible a outliers |  Sí, mucho                           |  No, al trabajar con rangos                                 |
| Escala utilizada    | Escala original                       | Rangos (datos convertidos a posiciones)                      |


**Comparación visual entre ambas matrices**

* Matriz de Pearson:

Se observan correlaciones muy altas (casi 1) entre algunas variables como BodyWgt y BrainWgt.

También aparecen correlaciones negativas marcadas entre variables como Gest y Sleep.

Sin embargo, la intensidad puede estar amplificada por outliers o por relaciones no perfectamente lineales.

* Matriz de Spearman:

Las correlaciones siguen una tendencia similar, pero son más moderadas (es decir, no tan extremas).

Las relaciones como BrainWgt ~ BodyWgt siguen siendo fuertes, pero no perfectas.

Algunas correlaciones negativas están más suavizadas, lo cual puede reflejar que las relaciones no son lineales, pero sí monótonas.

                                                                                             
- **Lineal**:

Cuando los datos siguen aproximadamente una **línea recta**. Es decir, por cada incremento en X, Y cambia siempre en proporción constante.                                          

- **Monótona**:

Cuando una variable **siempre aumenta o disminuye con la otra**, pero no necesariamente en línea recta. Puede crecer más rápido, más lento.


**Podemos responder entonces:**

- ¿Hay colinealidad?
- ¿Qué variables parecen tener una relación no lineal?
- ¿Conviene usar Spearman si hay outliers?

### Covarianza:

In [22]:
# Paso 1: Convertimos la matriz de covarianza en data frame y la reestructuramos en formato largo





In [23]:
# Paso 2: Visualizamos la matriz de covarianza como un mapa de calor con valores incluidos






¿Qué vemos?:

El valor numérico de la covarianza entre dos variables (p. ej. entre BodyWgt y BrainWgt) está asociado con un color rojo más intenso, indicando mayor covarianza positiva.

Los valores más cercanos a 0 (blanco o rosa muy claro) indican poca o nula relación lineal conjunta.


1. Covarianzas altas y positivas:

* BodyWgt con BrainWgt

* BodyWgt con Gest

* BrainWgt con Gest


Estas variables tienden a variar conjuntamente: cuando una aumenta, las otras también. Esto tiene sentido biológico, ya que animales más grandes (mayor BodyWgt) tienden a tener cerebros más grandes (BrainWgt) y gestaciones más largas (Gest).

2. Covarianzas negativas

* BodyWgt con Sleep
* BodyWgt con NonD
* Gest con Sleep


Estas relaciones indican que cuando una variable aumenta, la otra tiende a disminuir.

Por ejemplo:

Los animales más grandes tienden a dormir menos.
Los animales con mayor gestación también duermen menos (posiblemente especies más longevas, activas o expuestas).

3. Covarianzas cercanas a cero

* Dream con casi todas las demás variables.
* NonD con Dream, Gest, etc.


No hay una relación clara ni fuerte entre estas variables en términos de variación conjunta.

Es **importante* tener en cuenta que la covarianza depende de las unidades de las variables, por eso hay números muy grandes (como BrainWgt en gramos, por ejemplo).

Para comparar más fácilmente entre pares de variables, es mejor usar la correlación, que está normalizada entre -1 y 1.

Esta matriz sí es muy útil para detectar agrupaciones de variables altamente relacionadas, como BodyWgt, BrainWgt, Gest.

## Tablas de frecuencias y análisis de variables categóricas


En esta parte del módulo, nos enfocaremos en reconocer y analizar variables categóricas nominales y ordinales.

Para ello, elaboraremos tablas de frecuencias (absolutas y relativas).

Vamos a representar visualmente la distribución de variables categóricas.

Y exploraremos asociaciones entre variables categóricas (introducción a tablas de contingencia).

In [3]:
# Cargar el paquete janitor, que incluye funciones útiles para el manejo de tablas categóricas

# Tabla de frecuencias absolutas y relativas para la variable 'Danger'





# Tabla de frecuencias para la variable 'Pred'






# Tabla de frecuencias para la variable 'Exp'







In [ ]:
# Gráfico de barras de la variable Danger




In [ ]:
# Gráfico de barras de la variable Danger ordenado descendentemente





In [ ]:
# Para graficar un pie-chart
# ggplo2 no ofrece directamente un pie-chart, por lo que se deben hacer algunos cálculos
# Paso 1: Calcular la tabla de frecuencias






# Paso 2: Crear el gráfico de pastel




  # Agregar etiquetas de porcentaje centradas en cada porción




  # Agregar títulos y leyenda




  # Modificar el tamaño del título y leyenda





In [ ]:
#Diagarama de barras de la bariable Exp ordenado por frecuencia descendente




In [ ]:
#Diagrama de frecuencias variable Pred





In [ ]:
#Para poder agregar la etiqueta de los porcentajes:

# Paso 1: Calcular las frecuencias relativas





# Paso 2: Gráfico con etiquetas






### Tablas cruzadas

In [ ]:
# Tabla cruzada entre dos variables categóricas
# Tabla cruzada con porcentajes por fila y renombrado








In [ ]:
#Diagrama de barras con dos variables categóricas e inclusión de la frecuencia

# Paso 1: Calcular proporciones por grupo





# Paso 2: Gráfico de barras apiladas con etiquetas









In [ ]:
# Análisis de asociación o relación entre dos variables categóricas






* Si el valor-p es bajo (generalmente < 0.05):

Hay evidencia para rechazar la hipótesis nula, por lo que las variables sí están asociadas.

* Si el valor-p es alto:

No rechazamos la hipótesis nula, por lo que no hay suficiente evidencia para concluir que hay una relación, por lo que las variables parecen independientes.



El test de Chi-cuadrado asume que:

Al menos el 80% de las celdas deben tener frecuencias esperadas ≥ 5.

Ninguna celda debería tener una frecuencia esperada menor que 1.

Si esto no se cumple, la aproximación al valor-p usando la distribución Chi² puede no ser confiable, y por eso R nos advierte.



## Distribuciones de probabilidad


¿Qué es una distribución de probabilidad?

Una distribución de probabilidad describe cómo se distribuyen los valores posibles de una variable aleatoria. Es un modelo matemático que asigna una probabilidad a cada valor (discreto) o rango de valores (continuo).

Para variables discretas, usamos funciones de masa de probabilidad (como binomial o Poisson).
Para variables continuas, usamos funciones de densidad (como la normal o exponencial).






### Distribución normal:

Es una de las más comunes y útiles en estadística:

Tiene forma de campana simétrica.

Se define por dos parámetros:

* Media (μ)
* Desviación estándar (σ)

**Ejemplo:**

Comparar la distribución empírica de una variable cuantitativa con la teórica normal, usando ggplot2.

Usaremos la variable log_BodyWgt (peso logarítmico), ya que anteriormente mostramos que la transformación mejora la simetría y reduce outliers.

In [ ]:
# Crear gráfico con histograma y curva de densidad normal teórica



  # Histograma con densidad (no frecuencia)




  # Curva de densidad normal teórica ajustada a los datos





  # Etiquetas y estilo






--------------
| Función   | ¿Qué hace?                       | Ejemplo práctico                |
| --------- | -------------------------------- | ------------------------------- |
| `rnorm()` | Genera datos aleatorios          | Simulación de muestras          |
| `dnorm()` | Densidad (altura de la curva)    | Dibujar curva normal            |
| `pnorm()` | Probabilidad acumulada           | Calcular $P(X < x)$             |
| `qnorm()` | Cuantil inverso (valor de corte) | Buscar z tal que $P(X < z) = p$ |

-------------

**Ejercicio**:

Se sabe que el tiempo promedio que duerme un mamífero adulto es de 10 horas al día, con una desviación estándar de 2 horas.
Se asume que los tiempos de sueño siguen una distribución normal.


Grafica la distribución normal con μ=10 y σ=2.

Calcula las siguientes probabilidades:
* Que un mamífero duerma más de 12 horas al día.
* Que un mamífero duerma entre 8 y 12 horas.
* Que un mamífero duerma menos de 6 horas.

Visualiza las áreas bajo la curva correspondientes a cada probabilidad.

Interpreta los resultados.

In [4]:
# Parámetros de la distribución
mu <- 10      # media
sigma <- 2    # desviación estándar

# Secuencia de valores x para graficar la curva
x_vals <- seq(0, 20, by = 0.1)

# Crear curva de densidad




# Cálculo de las probabilidades:
# a) P(X > 12)




# b) P(8 < X < 12)




# c) P(X < 6)




# Resultados




### Distribución Binomial

Modela el número de éxitos en un número fijo de ensayos independientes con dos posibles resultados: éxito o fracaso.

**Ejemplo típico:**

Lanzar una moneda 10 veces.
Probabilidad de obtener 6 caras.

Parámetros de esta distribución:
* n = número de ensayos
* p = probabilidad de éxito

In [ ]:
# Distribución binomial: 20 ensayos, probabilidad de éxito 0.3





# Gráfico de barras de la distribución







-------------------
| Función                 | Qué hace                                              | Ejemplo                              |
| ----------------------- | ----------------------------------------------------- | ------------------------------------ |
| `rbinom(n, size, prob)` | Simula datos binomiales (n muestras de tamaño `size`) | `rbinom(10, size = 20, prob = 0.5)`  |
| `dbinom(x, size, prob)` | Probabilidad exacta $P(X = x)$                        | `dbinom(5, size = 10, prob = 0.5)`   |
| `pbinom(q, size, prob)` | Probabilidad acumulada $P(X \leq q)$                  | `pbinom(6, 10, 0.5)` → $P(X \leq 6)$ |
| `qbinom(p, size, prob)` | Cuantil: valor $x$ tal que $P(X \leq x) = p$          | `qbinom(0.95, 10, 0.5)`              |

-------------


**Ejercicio**

Un grupo de biólogos está evaluando la peligrosidad de ciertas especies animales. De acuerdo con registros previos, aproximadamente el 40% de los animales evaluados son considerados peligrosos. Se eligen 20 animales al azar.

Queremos modelar este proceso como un experimento binomial con: n=20 ensayos (animales seleccionados)
y p=0.40 la probabilidad de éxito (que un animal sea peligroso)

Para ello, simula la distribución binomial correspondiente al número de animales peligrosos esperados en una muestra de 20.

Visualiza la distribución usando un gráfico de barras.

Calcula la probabilidad de que:
* Exactamente 8 animales sean peligrosos.
* Menos de 5 animales sean peligrosos.
* Al menos 10 animales sean peligrosos.
Interpreta los resultados.

In [5]:
# Parámetros
n <- 20      # número de ensayos
p <- 0.40    # probabilidad de éxito

# Crear distribución binomial
binom_df <- data.frame(
  x = 0:n,
  prob = dbinom(0:n, size = n, prob = p)
)

# Visualización





# Cálculos de probabilidades específicas





# Mostrar resultados






### Distribución de Poisson

Modela el número de eventos que ocurren en un intervalo de tiempo o espacio cuando son raros y aleatorios.

**Ejemplo típico:**

* Número de llamadas por minuto a un centro de atención.
* Número de accidentes en una carretera por día.

Parámetro:

* λ = número medio de eventos en el intervalo.
* Si, X∼Poisson(λ), entonces la esperanza y varianza son iguales a λ.

In [ ]:
# Distribución de Poisson con media 5







--------------------

| Función            | Qué hace                                               | Ejemplo                  |
| ------------------ | ------------------------------------------------------ | ------------------------ |
| `rpois(n, lambda)` | Simula conteos con media $\lambda$                     | `rpois(10, lambda = 4)`  |
| `dpois(x, lambda)` | Probabilidad exacta $P(X = x)$                         | `dpois(2, lambda = 3)`   |
| `ppois(q, lambda)` | Probabilidad acumulada $P(X \leq q)$                   | `ppois(4, lambda = 2.5)` |
| `qpois(p, lambda)` | Cuantil: valor mínimo $x$ tal que $P(X \leq x) \geq p$ | `qpois(0.9, lambda = 3)` |


--------------------


**Ejercicio**:

Un investigador estudia el número de despertares breves que tienen ciertos animales durante el sueño. Se ha observado que, en promedio, ocurren 3 despertares por noche.

Queremos modelar este proceso como una distribución de Poisson, con: λ=3 número medio de eventos por unidad de tiempo (noche)


Crea la distribución de Poisson con λ=3 para los valores posibles de 0 a 10 despertares.

Visualiza la distribución mediante un gráfico de barras.

Calcula las siguientes probabilidades:

* Que haya exactamente 2 despertares.
* Que haya 4 o más despertares.
* Que haya entre 1 y 3 despertares (inclusive).

Interpreta los resultados.

In [6]:
# Parámetro de la distribución
lambda <- 3

# Valores posibles de X (número de despertares)
x_vals <- 0:10

# Crear tabla de probabilidades
poisson_df <- data.frame(
  x = x_vals,
  prob = dpois(x_vals, lambda = lambda)
)



# Cálculo de las probabilidades:

# a) P(X = 2)





# b) P(X ≥ 4)





# c) P(1 ≤ X ≤ 3)





# Mostrar resultados






### Distribución exponencial:

La distribución exponencial es usada para modelar el tiempo entre eventos en un proceso que ocurre de manera aleatoria y continua a una tasa constante.

Tiene un solo parámetro: λ (lambda) = tasa de ocurrencia de eventos por unidad de tiempo

- **Media**: 1/λ

- **Varianza**: 1/λ^2


In [7]:
set.seed(123)  # reproducibilidad

# Simular 200 tiempos entre eventos con tasa lambda = 0.5
# (es decir, en promedio un evento cada 2 unidades de tiempo)
exp_data <- rexp(n = 200, rate = 0.5)

# Crear dataframe
exp_df <- data.frame(tiempo = exp_data)


#Crea el histograma para el tiempo





--------------------
| Función         | Qué hace                                     | Ejemplo                 |
| --------------- | -------------------------------------------- | ----------------------- |
| `rexp(n, rate)` | Simula datos de tiempos de espera            | `rexp(10, rate = 0.2)`  |
| `dexp(x, rate)` | Densidad: valor de $f(x)$                    | `dexp(3, rate = 0.5)`   |
| `pexp(q, rate)` | Probabilidad acumulada $P(X \leq q)$         | `pexp(4, rate = 0.2)`   |
| `qexp(p, rate)` | Cuantil: valor $x$ tal que $P(X \leq x) = p$ | `qexp(0.9, rate = 0.2)` |

--------------------

Ejercicio:

Simula una variable que represente los tiempos entre visitas a una estación de observación de animales. Supón que los eventos ocurren a una tasa constante de 0.25 visitas por hora.

Simula 300 observaciones usando una distribución exponencial con rate = 0.25.

Grafica el histograma junto con la curva teórica de densidad.

Calcula la media y la varianza observadas.

Compara tus resultados con los valores teóricos.

Ajusta un nuevo valor de lambda y repite los pasos anteriores. ¿Cómo cambia la distribución?

In [8]:
# Paso 1: Simular
set.seed(456)
tiempos <- rexp(n = 300, rate = 0.25)
df_tiempos <- data.frame(tiempo = tiempos)

# Paso 2: Visualizar



# Paso 3: Estadísticos





In [9]:
# valor distinto de la tasa:

# Paso 1: Simular
set.seed(456)
tiempos <- rexp(n = 300, rate = 0.75)
df_tiempos <- data.frame(tiempo = tiempos)

# Paso 2: Visualizar




# Paso 3: Estadísticos





# Evaluación final



In [ ]:
# ================================
# EJERCICIOS CON EL CONJUNTO DE DATOS DE DIABETES
# ================================

# Cargar librerías necesarias
# (Ya deberían estar instaladas si vienes de la sección anterior)
library(VIM)
library(dplyr)
library(ggplot2)
library(tidyr)
library(tibble)
library(mlbench)
library(forcats)
library(janitor)

# ===================================
# EJERCICIO 1: Cargar y explorar el dataset
# ===================================
# Carga el dataset `PimaIndiansDiabetes` y explóralo con `glimpse()`

# Código base:
data(PimaIndiansDiabetes)
df <- as.data.frame(PimaIndiansDiabetes)

# TU CÓDIGO:
# glimpse( ... )


# ===================================
# EJERCICIO 2: Limpieza de datos
# ===================================
# Reemplaza los valores 0 por NA en las variables: glucose, pressure y mass

# TU CÓDIGO:
# df <- df %>%
#   mutate(
#     glucose = ...,
#     pressure = ...,
#     mass = ...
#   )


# ===================================
# EJERCICIO 3: Imputación de datos con ridge
# ===================================
# Imputa los valores faltantes de las variables numéricas usando ridge regression
# Usa la función `empire::estimate_ridge`

# Selección previa:
df_numerico <- df %>% select_if(is.numeric)

# TU CÓDIGO:
# df_imputado <- empire::estimate_ridge(...)


# ===================================
# EJERCICIO 4: Filtrado de mujeres embarazadas
# ===================================
# Filtra el dataset para quedarte solo con mujeres con al menos 1 embarazo

# TU CÓDIGO:
# df_embarazadas <- df %>%
#   filter(...)


# ===================================
# EJERCICIO 5: Crear variable estandarizada de BMI
# ===================================
# Crea una nueva variable `mass_z` que estandarice la variable `mass` (z-score)

# TU CÓDIGO:
# df_embarazadas <- df_embarazadas %>%
#   mutate(mass_z = ...)


# ===================================
# EJERCICIO 6: Estadística descriptiva de glucosa
# ===================================
# Calcula la media, mediana, desviación típica, mínimo y máximo de `glucose`

# TU CÓDIGO:
# df_embarazadas %>%
#   summarise(
#     media = ...,
#     mediana = ...,
#     sd = ...,
#     min = ...,
#     max = ...
#   )


# ===================================
# EJERCICIO 7: Histograma con curva normal
# ===================================
# Dibuja el histograma de glucosa con densidad y superpone la curva normal teórica

# TU CÓDIGO:
# ggplot(...) +
#   geom_histogram(...) +
#   stat_function(...) +
#   labs(...)


# ===================================
# EJERCICIO 8: Correlación entre glucosa y masa
# ===================================
# Calcula la correlación entre `glucose` y `mass`, ignorando los NA

# TU CÓDIGO:
# cor(..., use = "complete.obs")


# ===================================
# EJERCICIO 9: Crear grupos de edad
# ===================================
# Crea una variable `grupo_edad` con cortes: [20,30), [30,40), [40,50), [50,60)

# TU CÓDIGO:
# df_embarazadas <- df_embarazadas %>%
#   mutate(grupo_edad = cut(...))


# ===================================
# EJERCICIO 10: Tabla cruzada
# ===================================
# Crea una tabla cruzada entre `diabetes` y `grupo_edad`

# TU CÓDIGO:
# table(..., ...)


# ===================================
# EJERCICIO 11: Unión con tabla externa
# ===================================
# Usa la tabla `recomendaciones` para añadir una columna con la recomendación
# de salud según grupo de edad y diabetes. Usa `left_join`.

recomendaciones <- tibble(
  grupo_edad = c("[20,30)", "[30,40)", "[40,50)", "[50,60)", "[60,90)"),
  diabetes = c("pos", "neg", "pos", "neg", "pos"),
  recomendacion = c(
    "Revisión semestral",
    "Promover actividad física",
    "Controles nutricionales",
    "Revisión anual",
    "Seguimiento médico mensual"
  )
)

# TU CÓDIGO:
# df_embarazadas <- df_embarazadas %>%
#   left_join(...)

# Visualizar resultado:
# View(df_embarazadas)

